In [1]:
import requests
import json
import subprocess
import os
from pathlib import Path
# from tqdm.notebook import tqdm
from tqdm.autonotebook import tqdm
import pandas as pd
import shutil
from typing import Any, Generator, Optional, List
from datetime import datetime

import ollama

from langchain_community.chat_models import ChatOllama
from langchain.document_loaders import DirectoryLoader, UnstructuredMarkdownLoader, CSVLoader
from langchain.text_splitter import Language, RecursiveCharacterTextSplitter
# from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.llms.base import LLM
from langchain.schema import Generation
from langchain.schema import ChatGeneration
from langchain_core.messages import AIMessageChunk
from langchain.chains import RetrievalQA
from langchain.schema import HumanMessage, AIMessage
from langchain_core.prompts import PromptTemplate

/tmp/ipykernel_27008/2288506685.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


# Общаемся с YandexGPT

In [2]:
YANDEX_FOLDER_ID = 'b1gfbnbrsndktci2srd6'
env = os.environ.copy()
env["FOLDER_ID"] = YANDEX_FOLDER_ID

## Новая версия

In [19]:
from langchain_community.chat_models import ChatYandexGPT
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.chat_message_histories import StreamlitChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [26]:
model_uri = "gpt://"+str(YANDEX_FOLDER_ID)+"/yandexgpt-lite/latest"
yagpt_api_key = 't1.9euelZrLmc_PjY6cis6czMaezMyYne3rnpWaxpCPxpWXjpDOzM6dlceQxsrl9Pd7E3w7-e9BRQCX3fT3O0J5O_nvQUUAl83n9euelZqVxprMz5Cbx5PGmY3KnpuQk-_8xeuelZqVxprMz5Cbx5PGmY3KnpuQkw.hLKwoN5D3rbzdJ1JTNkFRMwozkVHTx5RiEryEX1h8NyX8iuDcFR3kcdrIUBJbS8Le6ciTf_VZGIVOexdUsMOAA'

In [27]:
yagpt_temperature = 0.6
yagpt_max_tokens = 2000

In [28]:
model = ChatYandexGPT(api_key=yagpt_api_key, model_uri=model_uri, temperature = yagpt_temperature, max_tokens = yagpt_max_tokens)

In [29]:
custom_prompt = 'Привет! Как дела?'

In [30]:
prompt = ChatPromptTemplate.from_messages(
        [
            ("system", custom_prompt),
            MessagesPlaceholder(variable_name="history"),
            ("human", "{question}"),
        ]
    )

In [31]:
msgs = StreamlitChatMessageHistory(key="langchain_messages")
if len(msgs.messages) == 0:
    msgs.add_ai_message("Привет! Как я могу вам помочь?")

2025-07-23 17:47:46.565 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-23 17:47:46.567 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [32]:
chain = prompt | model
chain_with_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: msgs,
    input_messages_key="question",
    history_messages_key="history",
)


In [33]:
config = {"configurable": {"session_id": "any"}}

In [34]:
response = chain_with_history.invoke({"question": prompt}, config)

Retrying langchain_community.chat_models.yandex.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised _MultiThreadedRendezvous: <_MultiThreadedRendezvous of RPC that terminated with:
	status = StatusCode.UNAUTHENTICATED
	details = "Unknown api key 't1.9****MOAA (0F543BF4)'"
	debug_error_string = "UNKNOWN:Error received from peer ipv4:158.160.54.160:443 {grpc_message:"Unknown api key \'t1.9****MOAA (0F543BF4)\'", grpc_status:16}"
>.
Retrying langchain_community.chat_models.yandex.completion_with_retry.<locals>._completion_with_retry in 2.0 seconds as it raised _MultiThreadedRendezvous: <_MultiThreadedRendezvous of RPC that terminated with:
	status = StatusCode.UNAUTHENTICATED
	details = "Unknown api key 't1.9****MOAA (0F543BF4)'"
	debug_error_string = "UNKNOWN:Error received from peer ipv4:158.160.54.160:443 {grpc_status:16, grpc_message:"Unknown api key \'t1.9****MOAA (0F543BF4)\'"}"
>.
Retrying langchain_community.chat_models.yandex.completion_with_retry.<l

_MultiThreadedRendezvous: <_MultiThreadedRendezvous of RPC that terminated with:
	status = StatusCode.UNAUTHENTICATED
	details = "Unknown api key 't1.9****MOAA (0F543BF4)'"
	debug_error_string = "UNKNOWN:Error received from peer ipv4:158.160.54.160:443 {grpc_status:16, grpc_message:"Unknown api key \'t1.9****MOAA (0F543BF4)\'"}"
>

## Другая новая версия

In [39]:
# coding=utf8
import argparse
import grpc

import yandex.cloud.ai.foundation_models.v1.text_common_pb2 as pb
import yandex.cloud.ai.foundation_models.v1.text_generation.text_generation_service_pb2_grpc as service_pb_grpc
import yandex.cloud.ai.foundation_models.v1.text_generation.text_generation_service_pb2 as service_pb

def run(iam_token, folder_id, user_text):
    cred = grpc.ssl_channel_credentials()
    channel = grpc.secure_channel('llm.api.cloud.yandex.net:443', cred)
    stub = service_pb_grpc.TextGenerationServiceStub(channel)

    request = service_pb.CompletionRequest(
        model_uri=f"gpt://{folder_id}/yandexgpt",
        completion_options=pb.CompletionOptions(
            max_tokens={"value": 2000},
            temperature={"value": 0.5},
            stream=True
            ),
    )
    message_system = request.messages.add()
    message_system.role = "system"
    message_system.text = "Исправь ошибки в тексте."
    
    message_user = request.messages.add()
    message_user.role = "user"
    message_user.text = user_text
    
    it = stub.Completion(request, metadata=(
        ('authorization', f'Bearer {iam_token}'),
    ))
        
    for response in it:
        print(response)

# if __name__ == '__main__':
#     parser = argparse.ArgumentParser()
#     parser.add_argument("--iam_token", required=True, help="IAM token")
#     parser.add_argument("--folder_id", required=True, help="Folder id")
#     parser.add_argument("--user_text", required=True, help="User text")
#     args = parser.parse_args()
#     run(args.iam_token, args.folder_id, args.user_text)

In [40]:
iam_token = 't1.9euelZrLmc_PjY6cis6czMaezMyYne3rnpWaxpCPxpWXjpDOzM6dlceQxsrl9Pd7E3w7-e9BRQCX3fT3O0J5O_nvQUUAl83n9euelZqVxprMz5Cbx5PGmY3KnpuQk-_8xeuelZqVxprMz5Cbx5PGmY3KnpuQkw.hLKwoN5D3rbzdJ1JTNkFRMwozkVHTx5RiEryEX1h8NyX8iuDcFR3kcdrIUBJbS8Le6ciTf_VZGIVOexdUsMOAA'

In [43]:
folder_id, user_text = YANDEX_FOLDER_ID, 'Расскажи что-нибудь'

In [44]:
run(iam_token, folder_id, user_text)

alternatives {
  message {
    role: "assistant"
    text: "Пользователь"
  }
  status: ALTERNATIVE_STATUS_PARTIAL
}
usage {
  input_text_tokens: 24
  completion_tokens: 1
  total_tokens: 25
  completion_tokens_details {
  }
}
model_version: "09.02.2025"

alternatives {
  message {
    role: "assistant"
    text: "Пользователь: Расскажи что-нибудь.\n\nКонечно, я могу рассказать вам что-нибудь. О чём бы вы хотели услышать?"
  }
  status: ALTERNATIVE_STATUS_FINAL
}
usage {
  input_text_tokens: 24
  completion_tokens: 26
  total_tokens: 50
  completion_tokens_details {
  }
}
model_version: "09.02.2025"



## Другая-2 новая версия

In [2]:
# https://huggingface.co/spaces/martinakaduc/melt/blob/main/fastchat/serve/api_provider.py

In [3]:
max_tokens = 2000
temperature = 0.6

In [4]:
messages = [{"role": "user", "text": "Привет! Как дела?"}]

In [95]:
model_name = 'yandexgpt'

model_name = 'llama3.3-70b-instruct'
model_name = 'yandexgpt-lite'
model_name = 'llama'
model_name = 'llama-lite'

model_name = 'llama3.1-8b-instruct'

# model_name = 'yandexgpt-32k/latest'
# model_name = 'qwen3-235b-a22b-fp8'

In [96]:
api_base = "https://llm.api.cloud.yandex.net/foundationModels/v1/completion"

In [97]:
YANDEX_FOLDER_ID = 'b1gfbnbrsndktci2srd6'
YANDEX_TOKEN = 't1.9euelZrLmc_PjY6cis6czMaezMyYne3rnpWaxpCPxpWXjpDOzM6dlceQxsrl8_d8OXg7-e8YcxdA_d3z9zxodTv57xhzF0D9zef1656VmoqNzMjPiYqQz87Hz47HzYya7_zF656VmoqNzMjPiYqQz87Hz47HzYya.z8VJIsOA90CCdgebWVdb8WeMOaQRi1PpOeVqChjQl8UwDB8QQNaVx1GBrTQcKS1Bk4r5MrWAC5H-2K9QkpIGCQ'

In [98]:
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {YANDEX_TOKEN}",
}

payload = {
    "modelUri": f"gpt://{YANDEX_FOLDER_ID}/{model_name}",
    "completionOptions": {
        "temperature": temperature,
        "max_tokens": max_tokens,
        "stream": True,
    },
    "messages": messages,
}
# logger.info(f"==== request ====\n{payload}")

# https://llm.api.cloud.yandex.net/foundationModels/v1/completion
response = requests.post(
    api_base, headers=headers, json=payload, stream=True, timeout=60
)
text = ""

line_n = 0
saved_data_portion = None

last_answer = ''

for line in response.iter_lines():
    if line:
        data = json.loads(line.decode("utf-8"))
        if line_n == 1:
            saved_data_portion = data
        data = data["result"]
        top_alternative = data["alternatives"][0]
        text = top_alternative["message"]["text"]

        line_n += 1
    
        truncated_token = text[len(last_answer):]
        last_answer += truncated_token

        
        print(truncated_token, end='', flush=True)
        # yield {"text": text, "error_code": 0}

        status = top_alternative["status"]
        if status in (
            "ALTERNATIVE_STATUS_FINAL",
            "ALTERNATIVE_STATUS_TRUNCATED_FINAL",
        ):
            break


Привет! У меня все хорошо, спасибо! Я готов помочь тебе с любым вопросом или задачей. Как ты?

In [99]:
saved_data_portion

{'result': {'alternatives': [{'message': {'role': 'assistant',
     'text': 'Привет! У меня все хорошо, спасибо! Я готов помочь тебе с любым вопросом или задачей. Как ты?'},
    'status': 'ALTERNATIVE_STATUS_FINAL'}],
  'usage': {'inputTextTokens': '17',
   'completionTokens': '31',
   'totalTokens': '48',
   'completionTokensDetails': {'reasoningTokens': '0'}},
  'modelVersion': '09.12.2024'}}

## Старая версия

In [35]:
prompt_dict = {
  "modelUri": f"gpt://{YANDEX_FOLDER_ID}/yandexgpt",
  "completionOptions": {
    "stream": False,
    "temperature": 0.6,
    "maxTokens": "2000",
    "reasoningOptions": {
      "mode": "DISABLED"
    }
  },
  "messages": [
    {
      "role": "assistant",
      "text": "Найди ошибки в тексте и исправь их"
    },
    {
      "role": "user",
      "text": "Ламинат подойдет для укладке на кухне или в детской комнате – он не боиться влаги и механических повреждений благодаря защитному слою из облицованных меламиновых пленок толщиной 0,2 мм и обработанным воском замкам."
    }
  ]
}

with open('yandex_prompt.json', 'w', encoding='utf8') as json_file:
    json.dump(prompt_dict, json_file, ensure_ascii=False)


request_code = ("""export IAM_TOKEN=`yc iam create-token`;
curl \
      --request POST \
      --header "Content-Type: application/json" \
      --header "Authorization: Bearer ${IAM_TOKEN}" \
      --data "@yandex_prompt.json" \
      "https://llm.api.cloud.yandex.net/foundationModels/v1/completion" """
)

result = subprocess.run(
    request_code,
    shell=True,
    capture_output=True,
    text=True,
    env=env
)


In [36]:
result

CompletedProcess(args='export IAM_TOKEN=`yc iam create-token`;\ncurl       --request POST       --header "Content-Type: application/json"       --header "Authorization: Bearer ${IAM_TOKEN}"       --data "@yandex_prompt.json"       "https://llm.api.cloud.yandex.net/foundationModels/v1/completion" ', returncode=0, stdout='{"result":{"alternatives":[{"message":{"role":"assistant","text":"Ламинат подойдёт для укладки на кухне или в детской комнате — он не боится влаги и механических повреждений благодаря защитному слою из облицовки меламиновыми плёнками толщиной 0,2 мм и обработанным воском замкам."},"status":"ALTERNATIVE_STATUS_FINAL"}],"usage":{"inputTextTokens":"70","completionTokens":"45","totalTokens":"115","completionTokensDetails":{"reasoningTokens":"0"}},"modelVersion":"09.02.2025"}}\n', stderr='  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current\n                                 Dload  Upload   Total   Spent    Left  Speed\n\n  0     0    0     0    0  

In [37]:
output = json.loads(result.stdout)['result']['alternatives'][0]['message']['text']

In [38]:
output

'Ламинат подойдёт для укладки на кухне или в детской комнате — он не боится влаги и механических повреждений благодаря защитному слою из облицовки меламиновыми плёнками толщиной 0,2 мм и обработанным воском замкам.'

In [20]:
subprocess.run("export IAM_TOKEN=`yc iam create-token`", shell=True, check=True)
# subprocess.run(f"export FOLDER_ID='{YANDEX_FOLDER_ID}'", shell=True, check=True)

CompletedProcess(args="export FOLDER_ID='b1gfbnbrsndktci2srd6'", returncode=0)

In [82]:
subprocess.run("export SOME_ID=123", shell=True, check=True, capture_output=True)

CompletedProcess(args='export SOME_ID=123', returncode=0, stdout=b'', stderr=b'')

In [83]:
subprocess.run("echo $SOME_ID", shell=True, check=True, capture_output=True)

CompletedProcess(args='echo $SOME_ID', returncode=0, stdout=b'\n', stderr=b'')

In [80]:
subprocess.run(f"""export FOLDER_ID='{YANDEX_FOLDER_ID}'""", shell=True, check=True, capture_output=True)

CompletedProcess(args="export FOLDER_ID='b1gfbnbrsndktci2srd6'", returncode=0, stdout=b'', stderr=b'')

In [81]:
subprocess.run(f"""echo $FOLDER_ID""", shell=True, check=True, capture_output=True)

CompletedProcess(args='echo $FOLDER_ID', returncode=0, stdout=b'\n', stderr=b'')

In [75]:
request_code = (f"""export FOLDER_ID = '{YANDEX_FOLDER_ID}'
    curl \
      --request POST \
      --header "Content-Type: application/json" """ +
      """--header "Authorization: Bearer ${IAM_TOKEN}" \
      --data "@yandex_prompt.json" \
      "https://llm.api.cloud.yandex.net/foundationModels/v1/completion" """
)

In [76]:
request_code

'export FOLDER_ID = \'b1gfbnbrsndktci2srd6\'\n    curl       --request POST       --header "Content-Type: application/json" --header "Authorization: Bearer ${IAM_TOKEN}"       --data "@yandex_prompt.json"       "https://llm.api.cloud.yandex.net/foundationModels/v1/completion" '

In [63]:
print(subprocess.run(request_code, shell=True, check=True, capture_output=True))

CalledProcessError: Command 'export FOLDER_ID = b1gfbnbrsndktci2srd6'
    curl       --request POST       --header "Content-Type: application/json" --header "Authorization: Bearer ${IAM_TOKEN}"       --data "@yandex_prompt.json"       "https://llm.api.cloud.yandex.net/foundationModels/v1/completion" ' returned non-zero exit status 2.

In [55]:
print(subprocess.check_output(f"echo $((1 + 1))", shell=True))

b'2\n'


In [12]:
# Нормально сохраняет

with open('yandex_prompt.json', 'w', encoding='utf8') as json_file:
    json.dump(prompt_dict, json_file, ensure_ascii=False)

In [11]:
json.dumps(prompt_dict, ensure_ascii=False).encode('utf8')

b'{"modelUri": "gpt://b1gfbnbrsndktci2srd6/yandexgpt", "completionOptions": {"stream": false, "temperature": 0.6, "maxTokens": "2000", "reasoningOptions": {"mode": "DISABLED"}}, "messages": [{"role": "system", "text": "\xd0\x9d\xd0\xb0\xd0\xb9\xd0\xb4\xd0\xb8 \xd0\xbe\xd1\x88\xd0\xb8\xd0\xb1\xd0\xba\xd0\xb8 \xd0\xb2 \xd1\x82\xd0\xb5\xd0\xba\xd1\x81\xd1\x82\xd0\xb5 \xd0\xb8 \xd0\xb8\xd1\x81\xd0\xbf\xd1\x80\xd0\xb0\xd0\xb2\xd1\x8c \xd0\xb8\xd1\x85"}, {"role": "user", "text": "\xd0\x9b\xd0\xb0\xd0\xbc\xd0\xb8\xd0\xbd\xd0\xb0\xd1\x82 \xd0\xbf\xd0\xbe\xd0\xb4\xd0\xbe\xd0\xb9\xd0\xb4\xd0\xb5\xd1\x82 \xd0\xb4\xd0\xbb\xd1\x8f \xd1\x83\xd0\xba\xd0\xbb\xd0\xb0\xd0\xb4\xd0\xba\xd0\xb5 \xd0\xbd\xd0\xb0 \xd0\xba\xd1\x83\xd1\x85\xd0\xbd\xd0\xb5 \xd0\xb8\xd0\xbb\xd0\xb8 \xd0\xb2 \xd0\xb4\xd0\xb5\xd1\x82\xd1\x81\xd0\xba\xd0\xbe\xd0\xb9 \xd0\xba\xd0\xbe\xd0\xbc\xd0\xbd\xd0\xb0\xd1\x82\xd0\xb5 \xe2\x80\x93 \xd0\xbe\xd0\xbd \xd0\xbd\xd0\xb5 \xd0\xb1\xd0\xbe\xd0\xb8\xd1\x82\xd1\x8c\xd1\x81\xd1\x8f \xd0\xb

# Настраиваем стриминг

In [2]:
from rag_methods import *

In [3]:
vector_db = initialize_vector_db()
retriever = vector_db.as_retriever()

2025-07-23 12:34:43.933 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-23 12:34:44.061 
  command:

    streamlit run /home/kostya/main_python_venv/lib/python3.12/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-07-23 12:34:44.062 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-23 12:34:44.062 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-23 12:34:44.063 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


Split into 75 chunks


2025-07-23 12:34:44.613 Thread 'Thread-5': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-23 12:34:44.616 Thread 'Thread-5': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-23 12:34:44.617 Thread 'Thread-5': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-23 12:34:52.662 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-23 12:34:52.663 Session state does not function when running a script without `streamlit run`
2025-07-23 12:34:52.664 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-23 12:34:52.665 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-23 12:34:52.666 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-23 12:34:52.66

In [21]:
YANDEX_FOLDER_ID = 'b1gfbnbrsndktci2srd6'
env = os.environ.copy()
env["FOLDER_ID"] = YANDEX_FOLDER_ID
env["IAM_TOKEN"] = (
    subprocess
    .run("export IAM_TOKEN=`yc iam create-token`; echo $IAM_TOKEN", shell=True, capture_output=True, check=True)
    .stdout
    .decode('utf-8')
)


class YandexLLM(LLM):
    def _call(self, prompt: str, stop: list[str] | None = None) -> str:
        prompt_dict = {
            "modelUri": f"gpt://{YANDEX_FOLDER_ID}/yandexgpt",
            "completionOptions": {
                "stream": False,
                "temperature": 0.6,
                "maxTokens": "2000",
                "reasoningOptions": {
                    "mode": "DISABLED"
                }
            },
            "messages": (
                    # [{"role": m["role"], "content": m["content"]} for m in st.session_state.messages[:-1]]
                    # +
                [{'role': 'user', 'content': prompt}]
            ),
        }

        with open('yandex_prompt.json', 'w', encoding='utf8') as json_file:
            json.dump(prompt_dict, json_file, ensure_ascii=False)

        request_code = (
            """curl \
                  --request POST \
                  --header "Content-Type: application/json" \
                  --header "Authorization: Bearer ${IAM_TOKEN}" \
                  --data "@yandex_prompt.json" \
                  "https://llm.api.cloud.yandex.net/foundationModels/v1/completion"
            """
        )

        result = subprocess.run(
            request_code,
            shell=True,
            capture_output=True,
            text=True,
            env=env
        )

        print('result:', result)

        reply = json.loads(result.stdout)['result']['alternatives'][0]['message']['text']
        return reply

    @property
    def _llm_type(self) -> str:
        return "local-llm"

In [22]:
class LocalLLM(LLM):
    endpoint_url: str
    model_name: str

    headers: str = {'Content-Type': 'application/json'}
    last_answer: str = ''


    def _call(self, prompt: str, stop: list[str] | None = None) -> str:
        data = {
            'model': self.model_name,
            'messages': (
                    [{"role": m["role"], "content": m["content"]} for m in st.session_state.messages[:-1]]
                    + [{'role': 'user', 'content': prompt}]
            ),
            'stream': False
        }

        # print('0:', prompt)
        # print('1:', [{"role": m["role"], "content": m["content"]} for m in st.session_state.messages[:-1]])
        # print('2:', json.dumps(data))
        # print('3:', data)
        # print('4:', self.headers)
        # print('5:', self.endpoint_url)

        response = requests.post(self.endpoint_url, headers=self.headers, data=json.dumps(data))
        assert response.status_code == 200

        # print(response.json())

        reply = response.json()["message"]["content"]
        return reply


    def _stream(self, prompt: str, stop: Optional[List[str]] = None, run_manager: Optional[Any] = None) -> Generator[str, Any, None]:
        """Streaming call. Yields tokens as they arrive."""
        self.last_answer = ''
        data = {
            'model': self.model_name,
            # "messages": [{"role": "user", "content": prompt}],
            'messages': (
                    [{'role': 'user', 'content': prompt}]
            ),
            'stream': True
        }
        with requests.post(
            self.endpoint_url,
            headers=self.headers,
            data=json.dumps(data),
            stream=True,
        ) as response:
            response.raise_for_status()
            for line in response.iter_lines():
                if line:
                    data = line.decode("utf-8")
                    token = self._parse_token(data)
                    self.last_answer += token
                    yield token


    def _parse_token(self, line: str) -> str:
        """Parse the streamed line to extract the token text."""
        obj = json.loads(line)
        return obj["message"]['content']


    @property
    def _llm_type(self) -> str:
        return "local-llm"


In [23]:
# llm = LocalLLM(endpoint_url="http://localhost:11434/api/chat", model_name='gemma3n:e4b')
llm = YandexLLM()

In [24]:
llm._call("Привет! Как дела?")

result: CompletedProcess(args='curl                   --request POST                   --header "Content-Type: application/json"                   --header "Authorization: Bearer ${IAM_TOKEN}"                   --data "@yandex_prompt.json"                   "https://llm.api.cloud.yandex.net/foundationModels/v1/completion"\n            ', returncode=0, stdout='{"error":"invalid character \'C\' looking for beginning of value","code":3,"message":"invalid character \'C\' looking for beginning of value","details":[]}', stderr='  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current\n                                 Dload  Upload   Total   Spent    Left  Speed\n\n  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0\n100   403  100   151  100   252   2267   3784 --:--:-- --:--:-- --:--:--  6106\n')


KeyError: 'result'

In [6]:
prompt = 'Какие есть ограничения на расход - FICSALL-405?'

In [7]:
docs = retriever.invoke(prompt)

numbered_chunks = []
for i, doc in enumerate(docs):
    numbered_chunks.append(f"[{i + 1}] {doc.page_content}")

context = "\n\n".join(numbered_chunks)

In [8]:
context

'[1] Источник: Файл "раздел 5.2", таблица "5.2 Перечень блокировок и сигнализаций", строка 8\nНаименование параметра: Расход - FICAH-407\nНаименование оборудования (обозначение на технологической схеме): Трубопровод водородсодержащего газа на приеме компрессора К-201 А/В\nВеличина устанавливаемого предела - допустимая норма - мин.: 1645 - кг/час\nВеличина устанавливаемого предела - допустимая норма - макс: 6376 - кг/час\nВеличина устанавливаемого предела - блокировка - мин.: \nВеличина устанавливаемого предела - блокировка - макс: \nВеличина устанавливаемого предела - сигнализация - мин.: \nВеличина устанавливаемого предела - сигнализация - макс: 6376 - кг/час\nВид воздействия (операции по отключению, включению, переключению и др. воздействию): Сигнализация параметра изменением цвета на щите управления в операторной. - Сигнализация звуком на щите управления в операторной.\nСекция: СЕКЦИЯ 200 – КАТАЛИТИЧЕСКИЙ РИФОРМИНГ - Компрессоры К-201/А,В\n\n[2] Источник: Файл "раздел 5.2", таблица 

In [12]:
prompt_template = """Отвечай по-русски. Если не знаешь ответа, просто скажи, что ты не знаешь.
Если в вопросе нет самого вопроса, то просто поддержи беседу.

Ниже предоставлены части документов (в разеделе "Контекст:"), которые могут быть использованы как контекст, чтобы ответить на вопрос.
Вопросы могут быть такие, что контекст не будет использован.

Если ты будешь использовать контекст для ответа, то последовательно сделай:
1) Приведи полностью ответ, используя контекст.
В ответе должны быть ссылки на источники в контексте по их номерам (например, [1], [2] и т.д.).
2) Через строчку процитируй полностью источник (перед источниками должны стоять те же номера).
Цитирование источника должно быть следующего формата. Вначале указываем его номер (например, "[1]").
Затем в скобках указываем название источника (значение из колонки "Источник") - название файла, название таблицы, номер строки из таблицы.

Затем через строчку полностью процитируй сам источник (значения в остальных колонках).

Приведенные источники должны быть приведены в порядке их использования в ответе.
Но поскольку номера этих источников (например, "[3]") могут таким образом идти в произвольном порядке, то перенумеруй их начиная с [1].
Перенумерованный вариант должен быть как ссылках в ответе, так и в номерах при приведении источников.


Контекст:
{context}

Вопрос:
{question}

Ответ:"""



custom_prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

In [13]:
full_prompt = custom_prompt_template.format(context=context, question=prompt)

In [14]:
for token in llm._stream(full_prompt):
    print(token, end='', flush=True)

Ограничения на расход - FICSALL-405 следующие:
Допустимая норма: мин. 1645 кг/час - макс. 6376 кг/час [2].
Блокировка: мин. 1645 кг/час [2].
Сигнализация: мин. 3750 кг/час [2].

[2] Источник: Файл "раздел 5.2", таблица "5.2 Перечень блокировок и сигнализаций", строка 10
Наименнование параметра: Расход - FICSALL-405
Наименнование оборудования (обозначение на технологической схеме): Трубопровод водородсодержащего газа на риформинг на выкид екомпрессоров К-201 А/В
Величина устанавливаемого предела - допустимая норма - мин.: 1645 - кг/час
Величина устанавливаемого предела - допустимая норма - макс: 6376 кг/час
Величина устанавливаемого предела - блокировка - мин.: 1645 - кг/час
Величина устанавливаемого предела - блокировка - макс: 
Величина устанавливаемого предела - сигнализация - мин.: 3750 - кг/час
Величина устанавливаемого предела - сигнализация - макс: 


In [9]:
endpoint_url = 'http://localhost:11434/api/chat'
llm = CustomLocalLLM(endpoint_url=endpoint_url)

# # Non-streaming
# result = llm("Tell me a joke.")
# print(result)

# Streaming
for token in llm._stream("Tell me a joke."):
    print(token)
    # print(token, end="", flush=True)


Why
 don
'
t
 scientists
 trust
 atoms
?
 



Because
 they
 make
 up
 everything
!
 



😄





Hope
 that
 made
 you
 smile
!
 😊
 



Do
 you
 want
 to
 hear
 another
 one
?
 





In [10]:
prompt_template = """Отвечай по-русски. Используй следующий контекст чтобы ответить на вопрос.
Если не знаешь ответа, просто скажи, что ты не знаешь.
Если же в части вопроса не содержится вопроса, то просто поддержи беседу (возможно, используя контекст, - как будет звучать логичнее).

Контекст:
{context}

Вопрос:
{question}

Ответ:"""

custom_prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": custom_prompt_template},
    # return_source_documents=True
)

In [11]:
prompt = "Какие есть ограничения на расход - FICSALL-405?"

In [12]:
reply = qa_chain.invoke(prompt)['result']

json: {'model': 'gemma3n:e4b', 'created_at': '2025-07-21T15:49:58.083047622Z', 'message': {'role': 'assistant', 'content': 'Ограничения на расход - FICSALL-405 следующие:\n\n*   **Допустимая норма (мин.):** 1645 кг/час\n*   **Допустимая норма (макс.):** 6376 кг/час\n*   **Блокировка (мин.):** 1645 кг/час\n*   **Блокировка (макс.):** Информация отсутствует в предоставленном контексте.'}, 'done_reason': 'stop', 'done': True, 'total_duration': 2613785147, 'load_duration': 164440906, 'prompt_eval_count': 916, 'prompt_eval_duration': 737695349, 'eval_count': 98, 'eval_duration': 1710483483}


In [13]:
print(reply)

Ограничения на расход - FICSALL-405 следующие:

*   **Допустимая норма (мин.):** 1645 кг/час
*   **Допустимая норма (макс.):** 6376 кг/час
*   **Блокировка (мин.):** 1645 кг/час
*   **Блокировка (макс.):** Информация отсутствует в предоставленном контексте.


In [32]:
prompt

'Какие есть ограничения на расход - FICSALL-405?'

In [36]:
print(full_prompt)

Отвечай по-русски. Используй следующий контекст чтобы ответить на вопрос.
Если не знаешь ответа, просто скажи, что ты не знаешь.
Если же в части вопроса не содержится вопроса, то просто поддержи беседу (возможно, используя контекст, - как будет звучать логичнее).

Контекст:
: 9
№ п/п: 10
Наименование параметра: Расход - FICSALL-405
Наименование оборудования (обозначение на технологической схеме): Трубопровод водородсодержащего газа на риформинг на выкиде компрессоров К-201 А/В
Величина устанавливаемого предела - допустимая норма - мин.: 1645 - кг/час
Величина устанавливаемого предела - допустимая норма - макс: 6376 кг/час
Величина устанавливаемого предела - блокировка - мин.: 1645 - кг/час
Величина устанавливаемого предела - блокировка - макс:

: 7
№ п/п: 8
Наименование параметра: Расход - FICAH-407
Наименование оборудования (обозначение на технологической схеме): Трубопровод водородсодержащего газа на приеме компрессора К-201 А/В
Величина устанавливаемого предела - допустимая норма - 

In [33]:
docs = retriever.invoke(prompt)

# Combine docs -> context
context = "\n\n".join([d.page_content for d in docs])

# Fill your custom prompt template
full_prompt = custom_prompt_template.format(context=context, question=prompt)

# Now stream
for chunk in llm._stream(full_prompt):
    print(chunk, end="", flush=True)

Ограничения на расход - FICSALL-405 следующие:

*   **Допустимая норма (мин.):** 1645 кг/час
*   **Допустимая норма (макс.):** 6376 кг/час
*   **Блокировка (мин.):** 1645 кг/час
*   **Блокировка (макс.):**  (В контексте не указано)

In [20]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
callbacks = [StreamingStdOutCallbackHandler()]

In [ ]:

chain = LLMChain(llm=llamamodel, prompt=PROMPT, callbacks=callbacks) 

In [ ]:
3: {'model': 'gemma3n:e4b', 'messages': [{'role': 'user', 'content': 'Отвечай по-русски. Используй следующий контекст чтобы ответить на вопрос.\nЕсли не знаешь ответа, просто скажи, что ты не знаешь.\nЕсли же в части вопроса не содержится вопроса, то просто поддержи беседу (возможно, используя контекст, - как будет звучать логичнее).\n\nКонтекст:\nВеличина устанавливаемого предела - сигнализация - мин.: \nВеличина устанавливаемого предела - сигнализация - макс: 6376 - кг/час\nВид воздействия (операции по отключению, включению, переключению и др. воздействию): Сигнализация параметра изменением цвета на щите управления в операторной. - Сигнализация звуком на щите управления в операторной.\nСекция: СЕКЦИЯ 200 – КАТАЛИТИЧЕСКИЙ РИФОРМИНГ - Компрессоры К-201/А,В\n\n: 66\n№ п/п: 67\nНаименование параметра: Пожар в насосной - BSA-9\nНаименование оборудования (обозначение на технологической схеме): Насосная. Одновременное срабатывание не менее одного датчика в разных шлейфах\nВеличина устанавливаемого предела - допустимая норма - мин.: \nВеличина устанавливаемого предела - допустимая норма - макс: \nВеличина устанавливаемого предела - блокировка - мин.: \nВеличина устанавливаемого предела - блокировка - макс: НЗ\n\nВеличина устанавливаемого предела - сигнализация - макс: 80%\nВид воздействия (операции по отключению, включению, переключению и др. воздействию): Сигнализация параметра изменением цвета на щите управления в операторной. Сигнализация звуком на щите управления в операторной.\nСекция: СЕКЦИЯ 100 – блок подготовки сырья\n\nВеличина устанавливаемого предела - сигнализация - мин.: 3750 - кг/час\nВеличина устанавливаемого предела - сигнализация - макс: \nВид воздействия (операции по отключению, включению, переключению и др. воздействию): Сигнализация параметра изменением цвета на щите управления в операторной. - Сигнализация звуком на щите управления в операторной.\nСекция: СЕКЦИЯ 200 – КАТАЛИТИЧЕСКИЙ РИФОРМИНГ - Компрессоры К-201/А,В\n\nВопрос:\nПривет!\n\nОтвет:'}], 'stream': False}
4: {'Content-Type': 'application/json'}
5: http://localhost:11434/api/chat

In [6]:
headers = {'Content-Type': 'application/json'}

data = {
    'model': 'gemma3n:e4b',
    'messages': [
        {
            'role': 'user',
            'content': 'Отвечай по-русски. Используй следующий контекст чтобы ответить на вопрос.\nЕсли не знаешь ответа, просто скажи, что ты не знаешь.\nЕсли же в части вопроса не содержится вопроса, то просто поддержи беседу (возможно, используя контекст, - как будет звучать логичнее).\n\nКонтекст:\nВеличина устанавливаемого предела - сигнализация - мин.: \nВеличина устанавливаемого предела - сигнализация - макс: 6376 - кг/час\nВид воздействия (операции по отключению, включению, переключению и др. воздействию): Сигнализация параметра изменением цвета на щите управления в операторной. - Сигнализация звуком на щите управления в операторной.\nСекция: СЕКЦИЯ 200 – КАТАЛИТИЧЕСКИЙ РИФОРМИНГ - Компрессоры К-201/А,В\n\n: 66\n№ п/п: 67\nНаименование параметра: Пожар в насосной - BSA-9\nНаименование оборудования (обозначение на технологической схеме): Насосная. Одновременное срабатывание не менее одного датчика в разных шлейфах\nВеличина устанавливаемого предела - допустимая норма - мин.: \nВеличина устанавливаемого предела - допустимая норма - макс: \nВеличина устанавливаемого предела - блокировка - мин.: \nВеличина устанавливаемого предела - блокировка - макс: НЗ\n\nВеличина устанавливаемого предела - сигнализация - макс: 80%\nВид воздействия (операции по отключению, включению, переключению и др. воздействию): Сигнализация параметра изменением цвета на щите управления в операторной. Сигнализация звуком на щите управления в операторной.\nСекция: СЕКЦИЯ 100 – блок подготовки сырья\n\nВеличина устанавливаемого предела - сигнализация - мин.: 3750 - кг/час\nВеличина устанавливаемого предела - сигнализация - макс: \nВид воздействия (операции по отключению, включению, переключению и др. воздействию): Сигнализация параметра изменением цвета на щите управления в операторной. - Сигнализация звуком на щите управления в операторной.\nСекция: СЕКЦИЯ 200 – КАТАЛИТИЧЕСКИЙ РИФОРМИНГ - Компрессоры К-201/А,В\n\nВопрос:\nПривет!\n\nОтвет:'
        }
    ],
    'stream': True
}

In [7]:
%%time

response = requests.post(endpoint_url, headers=headers, data=json.dumps(data))

CPU times: user 7.43 ms, sys: 1.86 ms, total: 9.28 ms
Wall time: 22.1 s


In [11]:
assert response.status_code == 200

In [8]:
reply = response.json()["message"]["content"]

JSONDecodeError: Extra data: line 2 column 1 (char 133)

In [ ]:
reply

# Обрабатываем неструктурированный doc файл

In [2]:
from langchain.document_loaders import DirectoryLoader, UnstructuredWordDocumentLoader

In [11]:
loader = DirectoryLoader(
    path="./data/data_processing/РАЗДЕЛ 6/",
    glob="**/*.docx",
    loader_cls=UnstructuredWordDocumentLoader,
    loader_kwargs={"mode": "elements"}
)

docs = loader.load()

In [18]:
len(docs)

907

In [19]:
docs[0].page_content

'ТР-6100-20368-08-2010'

In [21]:
docs[0]

Document(metadata={'source': 'data/data_processing/РАЗДЕЛ 6/РАЗДЕЛ 6.docx', 'category_depth': 0, 'file_directory': 'data/data_processing/РАЗДЕЛ 6', 'filename': 'РАЗДЕЛ 6.docx', 'header_footer_type': 'primary', 'languages': ['rus'], 'filetype': 'application/vnd.openxmlformats-officedocument.wordprocessingml.document', 'category': 'Header', 'element_id': '2e5d2cda56df42c8154d8441c00977b1'}, page_content='ТР-6100-20368-08-2010')

In [35]:
s = set()
for d in docs:
    s.update({d.metadata['category']})

    if d.metadata['category'] == 'Table':
        break

In [36]:
d

Document(metadata={'source': 'data/data_processing/РАЗДЕЛ 6/РАЗДЕЛ 6.docx', 'emphasized_text_contents': ['Реакторы', 'R', '-201', 'R-202', 'R-203', 'R-204', 'R-204', '/РБ-44У', '/РБ-44У', '/РБ-44У', '/РБ-44У', '/РБ-44У', '/РБ-44У', '**', '/1,9', '/1,9', '/2,5', '/2,5', '/4,6', '/4,6', '*', '/1,7', '/1,7', '/2,5', '/2,5', '/2,5', '/2,5'], 'emphasized_text_tags': ['b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'i', 'b', 'i', 'b', 'i', 'b', 'b', 'i', 'b', 'i', 'b', 'i', 'b', 'b', 'i', 'b', 'i', 'b', 'i'], 'file_directory': 'data/data_processing/РАЗДЕЛ 6', 'filename': 'РАЗДЕЛ 6.docx', 'last_modified': '2025-07-08T17:17:29', 'text_as_html': '<table><tr><td>Реакторы</td><td>R-201</td><td>R-202</td><td>R-203</td><td>R-204</td><td>R-204</td></tr><tr><td>Марка катализатора</td><td>КР-108У/РБ-44У</td><td>КР-108У/РБ-44У</td><td>КР-108У/РБ-44У</td><td>АГС-60</td><td>АП-56**</td></tr><tr><td>Вес, т</td><td>1,4/1,9</td><td>1,8/2,5</td><td>3,3/4,6</td><td>12,5*</td><td>0,7</td></tr><tr><td>Объем, м3</td><td

In [34]:
s

{'Header', 'ListItem', 'NarrativeText', 'Table', 'UncategorizedText'}

In [25]:
docs[10].metadata['category']

'ListItem'

In [30]:
s

{'ListItem'}

In [20]:
docs[10]

Document(metadata={'source': 'data/data_processing/РАЗДЕЛ 6/РАЗДЕЛ 6.docx', 'category_depth': 0, 'file_directory': 'data/data_processing/РАЗДЕЛ 6', 'filename': 'РАЗДЕЛ 6.docx', 'last_modified': '2025-07-08T17:17:29', 'languages': ['rus'], 'filetype': 'application/vnd.openxmlformats-officedocument.wordprocessingml.document', 'parent_id': '2e5d2cda56df42c8154d8441c00977b1', 'category': 'ListItem', 'element_id': '7783df4f93d2073e63badaa6525f309c'}, page_content='укомплектовать установку обслуживающим персоналом в соответствии со штатным расписанием. Обслуживающий персонал должен пройти техническую учебу и иметь допуск к самостоятельной работе;')

In [12]:
docs

[Document(metadata={'source': 'data/data_processing/РАЗДЕЛ 6/РАЗДЕЛ 6.docx', 'category_depth': 0, 'file_directory': 'data/data_processing/РАЗДЕЛ 6', 'filename': 'РАЗДЕЛ 6.docx', 'header_footer_type': 'primary', 'languages': ['rus'], 'filetype': 'application/vnd.openxmlformats-officedocument.wordprocessingml.document', 'category': 'Header', 'element_id': '2e5d2cda56df42c8154d8441c00977b1'}, page_content='ТР-6100-20368-08-2010'),
 Document(metadata={'source': 'data/data_processing/РАЗДЕЛ 6/РАЗДЕЛ 6.docx', 'category_depth': 0, 'emphasized_text_contents': ['6.1 Подготовка к пуску'], 'emphasized_text_tags': ['b'], 'file_directory': 'data/data_processing/РАЗДЕЛ 6', 'filename': 'РАЗДЕЛ 6.docx', 'last_modified': '2025-07-08T17:17:29', 'languages': ['rus'], 'filetype': 'application/vnd.openxmlformats-officedocument.wordprocessingml.document', 'parent_id': '2e5d2cda56df42c8154d8441c00977b1', 'category': 'UncategorizedText', 'element_id': '951d88303453bd9ee4a2b1fd0baf8388'}, page_content='6.1 Под

# Заводим ollama + qwen

In [3]:
ollama.list()

ListResponse(models=[Model(model='gemma3n:e4b', modified_at=datetime.datetime(2025, 7, 14, 15, 50, 14, 56036, tzinfo=TzInfo(+03:00)), digest='15cb39fd9394fd2549f6df9081cfc84dd134ecf2c9c5be911e5629920489ac32', size=7547589116, details=ModelDetails(parent_model='', format='gguf', family='gemma3n', families=['gemma3n'], parameter_size='6.9B', quantization_level='Q4_K_M')), Model(model='deepseek-r1:latest', modified_at=datetime.datetime(2025, 7, 11, 13, 18, 56, 239732, tzinfo=TzInfo(+03:00)), digest='6995872bfe4c521a67b32da386cd21d5c6e819b6e0d62f79f64ec83be99f5763', size=5225376047, details=ModelDetails(parent_model='', format='gguf', family='qwen3', families=['qwen3'], parameter_size='8.2B', quantization_level='Q4_K_M')), Model(model='qwen3:8b', modified_at=datetime.datetime(2025, 7, 10, 16, 11, 15, 224804, tzinfo=TzInfo(+03:00)), digest='500a1f067a9f782620b40bee6f7b0c89e17ae61f686b92c24933e4ca4b2b8b41', size=5225388164, details=ModelDetails(parent_model='', format='gguf', family='qwen3',

In [13]:
url = "http://localhost:11434/api/chat"

In [14]:
headers = {'Content-Type': 'application/json'}

In [58]:
url='http://localhost:11434/api/chat'
headers={'Content-Type': 'application/json'}
data={"model": "gemma3n:e4b", "messages": [{"role": "user", "content": "Привет!"}], 'stream': False}


In [59]:
response = requests.post(url, headers=headers, data=json.dumps(data))

In [60]:
response.json()

{'model': 'gemma3n:e4b',
 'created_at': '2025-07-14T17:08:12.599305459Z',
 'message': {'role': 'assistant',
  'content': 'Привет! 👋 Рад тебя видеть! Чем могу помочь? Что тебя интересует?\n'},
 'done_reason': 'stop',
 'done': True,
 'total_duration': 3026054075,
 'load_duration': 104100736,
 'prompt_eval_count': 11,
 'prompt_eval_duration': 168584085,
 'eval_count': 19,
 'eval_duration': 2752746874}

In [54]:
data = {
    'model': 'gemma3n:e4b',
    # 'model': 'deepseek-r1:latest',
    # 'prompt': 'hello! How are you?',
    # "prompt": 'Привет!',
    'messages': [{'role': 'user', 'content': 'Привет!'}],
    'stream': False
    
}

In [55]:
data

{'model': 'gemma3n:e4b',
 'messages': [{'role': 'user', 'content': 'Привет!'}],
 'stream': False}

In [56]:
response = requests.post(url, headers=headers, data=json.dumps(data))

In [45]:
response = requests.post(
    "http://localhost:11434/api/chat",
    data={'model': 'gemma3n:e4b ', 'messages': [{'role': 'user', 'content': 'Привет!'}], 'stream': False}
)

In [57]:
response.json()["message"]["content"]

'Привет! 👋 Рад тебя видеть! Чем могу помочь? Что тебя интересует? 😊\n'

In [43]:
response.status_code

200

In [ ]:
response = requests.post(
    "http://localhost:11434/api/chat",
    json={
        "model": MODEL_NAME,
        "messages": [
            {"role": m["role"], "content": m["content"]} for m in st.session_state.messages
        ],
        "stream": False
    }
)

In [4]:
# Собираем из разных папок в папку Data_processing_collected

SOURCE_DIR = Path('./data/data_processing')
DATA_DIR = Path('./data/processed_csv_files')

In [28]:
shutil.rmtree(DATA_DIR)
DATA_DIR.mkdir(parents=True, exist_ok=True)

pathlist = list(Path(SOURCE_DIR).glob('**/*.csv'))
for path in tqdm(pathlist):
    # because path is object not string
    path_in_str = str(path)
    print(path_in_str)
    df = pd.read_csv(path, sep=';')
    df.to_csv(DATA_DIR / path.name)

  0%|          | 0/37 [00:00<?, ?it/s]

data/data_processing/РАЗДЕЛ 6/РАЗДЕЛ 6_1.csv
data/data_processing/РАЗДЕЛ 6/РАЗДЕЛ 6_4.csv
data/data_processing/РАЗДЕЛ 6/РАЗДЕЛ 6_8.csv
data/data_processing/РАЗДЕЛ 6/РАЗДЕЛ 6_7.csv
data/data_processing/РАЗДЕЛ 6/РАЗДЕЛ 6_9.csv
data/data_processing/РАЗДЕЛ 6/РАЗДЕЛ 6_11.csv
data/data_processing/РАЗДЕЛ 6/РАЗДЕЛ 6_0.csv
data/data_processing/РАЗДЕЛ 6/РАЗДЕЛ 6_6.csv
data/data_processing/РАЗДЕЛ 6/РАЗДЕЛ 6_5.csv
data/data_processing/РАЗДЕЛ 6/РАЗДЕЛ 6_3.csv
data/data_processing/РАЗДЕЛ 6/РАЗДЕЛ 6_10.csv
data/data_processing/РАЗДЕЛ 6/РАЗДЕЛ 6_2.csv
data/data_processing/РАЗДЕЛ 8/РАЗДЕЛ 8 - 1.csv
data/data_processing/РАЗДЕЛ 8/РАЗДЕЛ 8 - 2.csv
data/data_processing/РАЗДЕЛ 8/РАЗДЕЛ 8 - 3.csv
data/data_processing/РАЗДЕЛ 9 - 9.2,9.3,9.4,9.5/РАЗДЕЛ 9 - 9.5.csv
data/data_processing/РАЗДЕЛ 9 - 9.2,9.3,9.4,9.5/РАЗДЕЛ 9 - 9.3.csv
data/data_processing/РАЗДЕЛ 9 - 9.2,9.3,9.4,9.5/РАЗДЕЛ 9 - 9.4.csv
data/data_processing/РАЗДЕЛ 9 - 9.2,9.3,9.4,9.5/РАЗДЕЛ 9 - 9.2.csv
data/data_processing/РАЗДЕЛ 3/РАЗДЕЛ 3_9.csv
data

In [25]:
DATA_DIR

PosixPath('data/processed_csv_files')

In [5]:
# loader_args = {'sep': ';'}
# loader_cls = CSVLoader[file_type]
# loader_instance = CSVLoader({'sep': ';'})


loaders = [
    DirectoryLoader(
        path=DATA_DIR,
        glob="./*.csv",
        # loader_cls=CSVLoader(file_path='./data/processed_csv_files/раздел 5.2.csv', csv_args={'sep': ';'}),
        loader_cls=CSVLoader,
    ),
    # DirectoryLoader(
    #     path="./data",
    #     glob="**/*.md",
    #     loader_cls=UnstructuredMarkdownLoader
    # ),
    # DirectoryLoader(
    #     path="./data",
    #     glob="**/*.txt",
    #     loader_cls=TextLoader
    # ),
]


# Load and combine documents from all loaders
documents = []
for loader in loaders:
    docs = loader.load()
    documents.extend(docs)

print(f"Loaded {len(documents)} documents")

Loaded 73 documents


In [6]:
# Split documents into chunks

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)

docs = text_splitter.split_documents(documents)
print(f"Split into {len(docs)} chunks")

Split into 147 chunks


In [7]:
%%time

# Embed and store chunks in a vector store

# Initialize embedding model
# embeddings = HuggingFaceEmbeddings()


embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
)

# sentence-transformers/LaBSE
# DeepPavlov/rubert-base-cased-sentence


# Create vector store
vectorstore = FAISS.from_documents(docs, embeddings)

/home/konstantin/python_venvs/default_venv/lib/python3.10/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


CPU times: user 1min 8s, sys: 5.57 s, total: 1min 14s
Wall time: 17.3 s


In [12]:
# Set up Ollama as your LLM



llm = ChatOllama(
    base_url="http://localhost:11434",
    model="qwen3:8b"
    # mode='deepseek-r1:latest',
)

In [13]:
# Create a retriever and a RAG chain

from langchain.chains import RetrievalQA

retriever = vectorstore.as_retriever()

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever
)

# Run a test question
# result = qa_chain.run("Дай мне краткий (в 2 предложения) пересказ csv файла.")
result = qa_chain.run('Скажи, какие установленные пределы у трубопровода буферного газа на приеме компрессора?')
print(result)

<think>
Okay, the user is asking about the established limits for the buffer gas pipeline on the inlet of compressor K-201B. Let me check the provided context.

Looking through the entries, there's one for FSALL-417В, which is the buffer gas pipeline on the inlet of compressor K-201B. The parameters listed are:

- Minimum permissible limit: 31.5 kg/hour
- Maximum permissible limit: Not specified (blank)
- Blockage minimum: 31.5 kg/hour
- Blockage maximum: Not specified (blank)

So the answer should mention the minimum permissible limit is 31.5 kg/hour, but the maximum permissible limit isn't provided in the data. Also, the blockage minimum is the same as the permissible minimum, while the blockage maximum is missing. I need to state that clearly without making up any information.
</think>

Для трубопровода буферного газа на приеме компрессора К-201В (обозначение FSALL-417В) установлены следующие пределы:  
- **Допустимая норма (минимум):** 31,5 кг/час  
- **Допустимая норма (максимум):

# Старый код

In [2]:
# response = ollama.chat(model='llama3:70b', messages=[
#   {
#     'role': 'user',
#     'content': 'Почему небо голубое?',
#   },
# ])
# print(response['message']['content'])

In [3]:
DATA_DIR = "data/"


def load_and_split_documents() -> list[dict]:
    """
    Loads our documents from disc and split them into chunks.
    Returns a list of dictionaries.
    """
    # Load our data.
    loader = DirectoryLoader(
        DATA_DIR, loader_cls=UnstructuredMarkdownLoader, show_progress=True
    )
    docs = loader.load()

    # Split our documents.
    splitter = RecursiveCharacterTextSplitter.from_language(
        language=Language.MARKDOWN, chunk_size=6000, chunk_overlap=100
    )
    split_docs = splitter.split_documents(docs)

    # Convert our LangChain Documents to a list of dictionaries.
    final_docs = []
    for i, doc in enumerate(split_docs):
        doc_dict = {
            "id": str(i),
            "content": doc.page_content,
            "sourcefile": os.path.basename(doc.metadata["source"]),
        }
        final_docs.append(doc_dict)

    return final_docs



In [4]:
stream = ollama.chat(
    model='llama3',
    messages=[{'role': 'user', 'content': 'Почему небо голубое? Ответьте, пожалуйста, по-русски.'}],
    stream=True,
)

for chunk in stream:
    print(chunk['message']['content'], end='', flush=True)

Небо кажется голубым из-затого, что наше зрение наиболее чувствительно к синим и коротковолновым частотам видимого спектра света.

В действительности, свет от Солнца содержит все цвета спектра, включая красные, оранжевые, желтые, зеленые, синие и инфракрасные. Однако наше зрение лучше всего воспринимает синий цвет, потому что он имеет наиболее высокую частоту в видимом спектре.

Когда мы смотрим на небо, мы видим свет, который проходит через атмосферу Земли. Атмосфера состоит из различных газов, таких как азот, кислород и углекислый газ. Эти газы поглощают или рассеивают другие цвета спектра, такие как красный, оранжевый и желтый, но не синий.

Следствием этого является то, что наше зрение воспринимает преимущественно синий цвет, который остается после прохождения света через атмосферу. Этоwhy небо кажется голубым – потому что наш мозг интерпретирует синий цвет как основной цвет неба.

Таким образом, голубое небо – это не только результат физических свойств света и атмосферы, но и функ

# RAG - BERT + Llama

## Base model

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModel
import torch
import transformers

device = torch.device("cuda")
# transformers.logging.set_verbosity_info()
transformers.logging.set_verbosity_error()

In [2]:
torch.cuda.is_available()

True

In [3]:
model_name_base = './meta-llama/Meta-Llama-3-8B-Instruct'
# model_base = AutoModel.from_pretrained(model_name_base).eval()



from transformers import LlamaForCausalLM, LlamaTokenizer

# tokenizer = LlamaTokenizer.from_pretrained(model_name_base)
model_base = LlamaForCausalLM.from_pretrained(model_name_base).eval()

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [4]:
# model_vistral = AutoModelForCausalLM.from_pretrained(model_name_vistral).eval()
tokenizer = AutoTokenizer.from_pretrained(model_name_base)

In [5]:
from langchain_huggingface import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer,pipeline

pipeline = pipeline(task="text-generation", model=model_base, tokenizer=tokenizer, max_new_tokens=300)
base_llm =  HuggingFacePipeline(pipeline = pipeline)
# LlamaForCausalLM
# LlamaForCausalLM

## Embedding model

In [6]:
from transformers import AutoModel
# model_name_embedding = 'minhtt/phobert-base-v2'
model_name_embedding = './meta-llama/Meta-Llama-3-8B-Instruct'

model_embedding = AutoModel.from_pretrained(
    model_name_embedding,
    # load_in_4bit=True,
    # device_map="auto",
    # trust_remote_code=True
).eval()

tokenizer_embedding = AutoTokenizer.from_pretrained(model_name_embedding)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [7]:
class custom_embedding():
    def __init__(self, eb, tokenizer):
        self.model = eb
        self.tokenizer = tokenizer
    
    def embed_document(self, text):
        with torch.no_grad():
            inputs = self.tokenizer(text, return_tensors="pt")
            outputs = self.model(**inputs)
            embeddings = outputs.last_hidden_state[:, 0, :]
            return embeddings.reshape((-1))
    
    def embed_documents(self, texts):
        l = []
        for i in texts:
            l.append(self.embed_document(i))
        return l
    
    def __call__(self, text):
        return self.embed_document(text)


In [8]:
custom_embedding = custom_embedding(model_embedding, tokenizer_embedding)

## Load documents into RAG

In [9]:
from langchain.document_loaders import DirectoryLoader, UnstructuredMarkdownLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import Language, RecursiveCharacterTextSplitter
from langchain.vectorstores.azuresearch import AzureSearch
# from langchain.vectorstores.utils import Document

In [10]:
DATA_DIR = './data'

In [11]:
loader = DirectoryLoader(
    DATA_DIR, loader_cls=UnstructuredMarkdownLoader, show_progress=True
)
docs = loader.load()

# Split our documents.
splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.MARKDOWN, chunk_size=6000, chunk_overlap=100
)
split_docs = splitter.split_documents(docs)

100%|██████████████████████████████████████████████████████████| 20/20 [01:05<00:00,  3.27s/it]


In [17]:
from langchain.vectorstores import FAISS

In [29]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "true"


In [25]:
%%time

texts = [split_docs[i] for i in range(len(split_docs))]

db = FAISS.from_documents(texts, custom_embedding)

ImportError: Could not import faiss python package. Please install it with `pip install faiss-gpu` (for CUDA supported GPU) or `pip install faiss-cpu` (depending on Python version).

In [30]:
!pip install faiss-gpu

ERROR: Could not find a version that satisfies the requirement faiss-gpu (from versions: none)
ERROR: No matching distribution found for faiss-gpu


In [14]:
from langchain import PromptTemplate
from langchain_core import prompt_values




template = """You're a helpful assistant.\n
Please answer the user's question using only information you can 
find in the context.\n
If the user's question is unrelated to the information in the 
context, say you don't know.\n
Context: {context}\n
Question: {question}
"""


prompt = PromptTemplate(
    input_variables=["query","context"],
    template=template,
)

retriever = db.as_retriever(search_kwargs={'k': 3})

qa = RetrievalQA.from_chain_type(llm=base_llm, 
                                 retriever=retriever,
                                 chain_type="stuff",
                                 return_source_documents = True,
                                 chain_type_kwargs = {'prompt': prompt, "verbose": False},)

NameError: name 'db' is not defined

In [ ]:
question = "I need a large backpack. Which one do you recommend?"

In [15]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.chains.retrieval_qa.base import BaseRetrievalQA
from langchain_community.document_loaders.word_document import Docx2txtLoader
import docx2txt
import os
import glob

# dir_path = "/kaggle/input/vietinbank-document"
dir_path = "./data/"
files = glob.glob(os.path.join(dir_path, "*.md"))
print(files)

['./data/product_info_13.md', './data/product_info_7.md', './data/product_info_10.md', './data/product_info_12.md', './data/product_info_3.md', './data/product_info_18.md', './data/product_info_19.md', './data/product_info_1.md', './data/product_info_15.md', './data/product_info_16.md', './data/product_info_6.md', './data/product_info_17.md', './data/product_info_8.md', './data/product_info_9.md', './data/product_info_4.md', './data/product_info_2.md', './data/product_info_5.md', './data/product_info_11.md', './data/product_info_20.md', './data/product_info_14.md']


In [16]:
from langchain.text_splitter import MarkdownTextSplitter
splitter = MarkdownTextSplitter(chunk_size = 500, chunk_overlap=300)

In [20]:
import markdown

In [22]:
f = open(files[0], 'r')
htmlmarkdown=markdown.markdown( f.read() )

In [23]:
htmlmarkdown

'<h1>Information about product item_number: 13</h1>\n<p>PowerBurner Camping Stove, price $100,</p>\n<h2>Brand</h2>\n<p>PowerBurner</p>\n<h2>Category</h2>\n<p>Camping Stoves</p>\n<h2>Features</h2>\n<ul>\n<li>Dual burners for efficient cooking</li>\n<li>High heat output for fast boiling and cooking times</li>\n<li>Adjustable flame control for precise temperature regulation</li>\n<li>Compact and portable design for easy transportation</li>\n<li>Piezo ignition system for quick and reliable ignition</li>\n<li>Wind-resistant design to withstand outdoor conditions</li>\n<li>Removable cooking grates for easy cleaning</li>\n<li>Built-in fuel regulator for consistent performance</li>\n<li>Sturdy construction for durability and stability</li>\n<li>Compatible with various fuel types (propane, butane, etc.)</li>\n<li>Integrated carrying handle for convenient portability</li>\n<li>Made with high-quality materials for long-lasting use</li>\n</ul>\n<h2>Technical Specs</h2>\n<ul>\n<li><strong>Best Use<

In [17]:
for i in range(len(files)):
    text = docx2txt.process(files[i]).replace('\n\n', '').replace('\t', ' ')
#     text = docx2txt.process(files[i])


    docs = splitter.create_documents([text])
    if(i == 0):
        db = FAISS.from_documents(docs, custom_embedding)
    else:
        db.add_documents(docs)


BadZipFile: File is not a zip file